# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションにおいて、AIエージェントが「20の質問」ゲームをプレイするための実装を行っています。具体的には、質問者エージェントと回答者エージェントを定義し、彼らが相互に作用しながらキーワードを推測する仕組みを構築しています。

主な取り組み:
- **データの読み込みと前処理**: `pandas`ライブラリを用いて、キーワードのリストをJSON形式で解析してデータフレームに変換する工程を行います。各カテゴリとその単語数を出力し、最終的にキーワードが整理されたデータフレームを生成します。
- **ライブラリのインストールとセットアップ**: `immutabledict`や`sentencepiece`など、必要なライブラリをインストールし、`gemma_pytorch`リポジトリをクローンしてエージェントが使用する準備を整えます。
- **エージェントクラスの実装**: 質問や回答を処理するための`GemmaQuestionerAgent`および`GemmaAnswererAgent`のクラスを定義します。これにより、各エージェントが会話の状態を管理し、質問や回答の履歴を追跡できます。
- **対話処理**: 各エージェントの役割に応じて、ユーザーからの質問に対する応答や、推測に対する反応を処理するための関数`agent_fn`を実装します。

使用ライブラリ:
- **pandas**: データフレームの作成と管理。
- **json**: JSONデータの解析。
- **torch**: 深層学習モデルの実行。
- **gemma**: 特定の因果関係を持つ言語モデルの使用を可能にするライブラリ。

このNotebookによって、AIエージェントが効率的に「20の質問」ゲームをプレイし、人間のプレイヤーと対話する能力を持つよう設計されています。エージェントは、キーワードを推測するための質問を生成し、提供された情報に基づいて合理的な推測を行うことが求められます。

---


# 用語概説 
このJupyter Notebookの内容を基に、機械学習・深層学習の初心者がつまずきそうな専門用語について簡単に解説します。以下の用語は、特にマイナーであったり、実務経験がない場合に馴染みが薄い可能性があります。

1. **Gemma**:
   - Gemmaは、Causal Language Model（因果言語モデル）の一種であり、特定のタスク（ここでは「20の質問」ゲーム）に対する質問生成や回答に特化しています。モデルの詳細やトレーニングには、特定のアーキテクチャやハイパーパラメータが関与します。

2. **Causal Language Model**:
   - 因果言語モデルは、これまでの文脈を基に次に来る単語を予測するモデルです。このモデルは、シーケンシャルなデータ（たとえば文章）を処理し、次に来る単語を推測するために使用されます。

3. **Quantization**:
   - モデルのサイズを小さくする技術で、特にメモリ使用量を削減する目的で、浮動小数点数を整数に変換します。これにより、計算効率が向上し、デプロイメントの際のリソース負荷が軽減されます。

4. **Few-shot examples**:
   - モデルがタスクを学習するために、わずかな数の例（few-shot）の入力を提供することを指します。これによりモデルは、トレーニングされていない状況でも新しいタスクについて推論できる能力を持つことが期待されます。

5. **Interleave**:
   - データやシーケンスを交互に組み合わせる操作を指します。例えば、2つのリストがある場合、それらを交互に合成して新しいリストを作成します。この操作は、質問と回答のペアを整理する際に利用されます。

6. **Context Manager**:
   - Pythonにおける構文で、特定のリソースを管理するための仕組みです。例えば、デフォルトのデータ型を一時的に変更する際に使用され、リソースの解放を自動的に行うことができます。

7. **Observation Object**:
   - ゲームの観察データを表すオブジェクトで、ゲームの状態（質問、回答、ターンタイプなど）を保持します。このオブジェクトは、エージェントが応答するために必要な情報を格納します。

8. **TurnType**:
   - ゲーム内でエージェントの役割（質問、回答、推測）を示す値です。この値により、エージェントの行動が異なるため、重要な役割を果たします。

9. **Regular Expression (re)**:
   - 特定のパターンを持つ文字列を検索、マッチング、置換するための表現法です。このノートブックでは、特定のキーワードを抽出するために使用されています。

10. **Pydantic**:
    - Pythonのデータバリデーションと設定管理のライブラリで、Pythonのクラスを基にしたデータモデルを簡単に作成し、データの整合性を確保します。ここでは、観察データを表す`Observation`クラスで使用されています。

このような専門用語の解説を提供することで、利用者がJupyter Notebookの内容を理解する手助けができると思います。

---


In [ ]:
# pandasライブラリをpdとしてインポートします
import pandas as pd
# jsonライブラリをインポートします
import json

# キーワードを読み込む
# 指定したパスからkeywords.pyファイルを動的に読み込みます
from importlib.machinery import SourceFileLoader
keywords = SourceFileLoader("keywords",'/kaggle/input/llm-20-questions/llm_20_questions/keywords.py').load_module()
# JSON形式のキーワードを解析し、Python辞書オブジェクトに変換します
df = json.loads(keywords.KEYWORDS_JSON)

# 単語を格納するためのリストを作成します
words = []
# 各カテゴリについてループします
for cat in df:
    # カテゴリ名とその単語数を出力します
    print(cat['category'], len(cat['words']))
    # 各カテゴリ内の単語についてループします
    for w in cat['words']:
        # カテゴリ名、キーワード、代替語、空欄のフィールド、緯度、経度をリストに追加します
        words.append([cat['category'], w["keyword"], w["alts"], "", "", 0.0, 0.0])

# キーワードをデータフレームに変換します
# 列名を指定してデータフレームを作成します
df = pd.DataFrame(words, columns=['Category','Word','Alternatives', "Cat1", "Cat2", "Lat", "Lon"])
# データフレームの先頭5行を表示します
df.head()

In [ ]:
%%bash
# 作業ディレクトリに移動します
cd /kaggle/working
# すべてのファイルとディレクトリを削除します
rm -rf ./*
# immutabledictとsentencepieceライブラリをインストールします
# -qオプションは出力を抑制し、-Uオプションはアップデートを実行します
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece
# gemma_pytorchリポジトリをクローンします
git clone https://github.com/google/gemma_pytorch.git > /dev/null
# gemma用のディレクトリを作成します
mkdir /kaggle/working/submission/lib/gemma/
# gemma_pytorchから必要なファイルを移動します
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/

In [ ]:
%%writefile submission/main.py
# # 準備
import os
import sys

# Kaggleエージェントのパスを指定します
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
# エージェントのパスが存在する場合、Libディレクトリをパスに追加します
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
else:
    # そうでない場合は、作業ディレクトリのLibディレクトリをパスに追加します
    sys.path.insert(0, "/kaggle/working/submission/lib")

import contextlib
import os
import sys
from pathlib import Path

import torch
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM

# ウェイトのパスを設定します
if os.path.exists(KAGGLE_AGENT_PATH):
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "gemma/pytorch/7b-it-quant/2")
else:
    WEIGHTS_PATH = "/kaggle/input/gemma/pytorch/7b-it-quant/2"


import itertools
from typing import Iterable


class GemmaFormatter:
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'

    # 初期化
    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()

    # 現在の会話状態を文字列として返します
    def __repr__(self):
        return self._state

    # ユーザープロンプトを会話に追加するメソッド
    def user(self, prompt):
        self._state += self._turn_user.format(prompt)
        return self
        
    # モデルのレスポンスを会話に追加するメソッド
    def model(self, prompt):
        self._state += self._turn_model.format(prompt)
        return self

    # ユーザーターンの開始をマークするメソッド
    def start_user_turn(self):
        self._state += f"{self._start_token}user\n"
        return self

    # モデルターンの開始をマークするメソッド
    def start_model_turn(self):
        self._state += f"{self._start_token}model\n"
        return self

    # 現在のターンの終了をマークするメソッド
    def end_turn(self):
        self._state += f"{self._end_token}\n"
        return self

    # リセットメソッド
    def reset(self):
        # `_state`を空の文字列で初期化します
        self._state = ""  

        # 提供された場合はシステムプロンプトを追加します
        if self._system_prompt is not None:
            self.user(self._system_prompt)  
            
        # 提供された場合はfew-shotの例を適用します
        if self._few_shot_examples is not None: 
            self.apply_turns(self._few_shot_examples, start_agent='user')
        return self

    def apply_turns(self, turns: Iterable, start_agent: str):
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)
        for fmt, turn in zip(formatters, turns):
            fmt(turn)
        return self


import re

@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    """デフォルトのtorchデータ型を指定されたdtypeに設定します。"""
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)

from pydantic import BaseModel
    
class Observation(BaseModel):
    step: int
    role: t.Literal["guesser", "answerer"]
    turnType: t.Literal["ask", "answer", "guess"]
    keyword: str
    category: str
    questions: list[str]
    answers: list[str]
    guesses: list[str]
    
    @property
    def empty(self) -> bool:
        return all(len(t) == 0 for t in [self.questions, self.answers, self.guesses])
    
    def get_history(self) -> t.Iterator[tuple[str, str, str]]:
        return itertools.zip_longest(self.questions, self.answers, self.guesses, fillvalue="[none]")

    def get_history_text(self, *, skip_guesses: bool = False, skip_answer: bool = False, skip_question: bool = False) -> str:
        if not self.empty:
            history = "\n".join(
            f"""{'**Question:** ' + question if not skip_question else ''} -> {'**Answer:** ' + answer if not skip_answer else ''}
            {'**Previous Guess:** ' + guess if not skip_guesses else ''}
            """
            for i, (question, answer, guess) in enumerate(self.get_history())
            )
            return history
        return "まだありません。"


class GemmaAgent:
    def __init__(self, variant='7b-it-quant', device='cuda:0', system_prompt=None, few_shot_examples=None):
        self._variant = variant
        self._device = torch.device(device)
        self.formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples)

        print("モデルを初期化しています")
        
        # モデルの設定
        model_config = get_config_for_2b() if "2b" in variant else get_config_for_7b()
        model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")
        model_config.quant = "quant" in variant
        
        # モデルを作成します
        with _set_default_tensor_type(model_config.get_dtype()):
            """TODO: パフォーマンスを見るために異なるモデルに変更します"""
            model = GemmaForCausalLM(model_config)
            ckpt_path = os.path.join(WEIGHTS_PATH , f'gemma-{variant}.ckpt')
            model.load_weights(ckpt_path)
            self.model = model.to(self._device).eval()

    def __call__(self, obs, *args):
        self._start_session(obs)
        prompt = str(self.formatter)
        response = self._call_llm(prompt)
        response = self._parse_response(response, obs)
        print(f"{response=}")
        return response

    def _start_session(self, obs: dict):
        raise NotImplementedError

    def _call_llm(self, prompt, max_new_tokens=32, **sampler_kwargs):
        """TODO: LLMのパラメータを変更します"""
        if sampler_kwargs is None:
            sampler_kwargs = {
                'temperature': 0.01,
                'top_p': 0.1,
                'top_k': 1,
        }
        response = self.model.generate(
            prompt,
            device=self._device,
            output_len=max_new_tokens,
            **sampler_kwargs,
        )
        return response

    def _parse_keyword(self, response: str):
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)
        if match is None:
            keyword = ''
        else:
            keyword = match.group().lower()
        return keyword

    def _parse_response(self, response: str, obs: dict):
        raise NotImplementedError


def interleave_unequal(x, y):
    return [
        item for pair in itertools.zip_longest(x, y) for item in pair if item is not None
    ]

"""TODO: 質問者を調整します"""
class GemmaQuestionerAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user("20の質問をプレイしましょう。あなたは質問者の役割を果たします。")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='model')
        hints = self.turns_to_hints(turns)
        if obs.turnType == 'ask':
            if len(turns) == 0:
                self.formatter.user("はいかいいえで答えられる質問をしてください。")
            else:
                self.formatter.user(f"以下のヒントに基づいて、はいかいいえの質問をしてください: {hints}")
        elif obs.turnType == 'guess':
            self.formatter.user("今、キーワードを推測してください。推測は二重アスタリスクで囲んでください。")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', ''))
            if match is None:
                question = "人ですか？"
            else:
                question = match.group()
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response)
            return guess
        else:
            raise ValueError("未知のターンタイプ:", obs.turnType)
    
    # 質問に基づいて肯定文または否定文に変換する
    def turns_to_hints(self, turns):
        hints = []
        for i in range(0, len(turns), 2):
            question = turns[i]
            answer = turns[i+1]
            # hints.append(f"Question: {question} Answer: {answer}")
            
            if question.startswith('is it'):
                item = question[6:-1]  # "is it "の後と"?"の前の部分を抽出します
                if answer == 'yes':
                    transformed_statement = f"それは{item}です"
                else:
                    transformed_statement = f"それは{item}ではありません"
            elif question.startswith('does'):
                item = question[5:-1]  # "does "の後と"?"の前の部分を抽出します
                if answer == 'yes':
                    transformed_statement = f"それは{item}をします"
                else:
                    transformed_statement = f"それは{item}をしません"
            else:
                transformed_statement = f"認識できない質問形式: {question}"

            hints.append(transformed_statement)

        return hints


class GemmaAnswererAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user(f"20の質問をプレイしましょう。あなたは回答者の役割を果たします。キーワードは「{obs.keyword}」で、カテゴリは「{obs.category}」です。")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='user')
        self.formatter.user(f"このキーワードについての質問は「{obs.keyword}」です。はいかいいえで答えて、答えは二重アスタリスクで囲んでください。")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        answer = self._parse_keyword(response)
        return 'yes' if 'yes' in answer else 'no'

# エージェント作成
system_prompt = """あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。 
このゲームでは、回答者がキーワードを考え、質問者からのはいかいいえの質問に答えます。 
キーワードは特定の人物、場所、または物です。あなたの目標は、できるだけ少ないターンで秘密のキーワードを推測して勝つことです。"""

few_shot_examples = [
    "20の質問をプレイしましょう。あなたは質問者の役割を果たします。最初の質問をしてください。",
    "人ですか?", "**いいえ**",
    "場所ですか?", "**はい**",
    "国ですか?", "**はい**推測してください。",
    "**フランス**", "正解です！",
]

# **重要:** エージェントをグローバルに定義して、必要なエージェントを一度だけ読み込みます。
# 両方を読み込むとOOMの原因になる可能性があります。

# エージェント変数を初期化します
agent = None

# 名前に基づいて適切なエージェントを取得する関数
def get_agent(name: str):
    global agent
    
    # エージェントが初期化されておらず、要求されたエージェントが「質問者」の場合
    if agent is None and name == 'questioner':
        # GemmaQuestionerAgentを特定のパラメータで初期化します
        agent = GemmaQuestionerAgent(
            device='cuda:0',  # 計算用デバイス
            system_prompt=system_prompt,  # エージェントのシステムプロンプト
            few_shot_examples=few_shot_examples,  # エージェントの行動を指導する例
        )
    # エージェントが初期化されておらず、要求されたエージェントが「回答者」の場合
    elif agent is None and name == 'answerer':
        # GemmaAnswererAgentを同じパラメータで初期化します
        agent = GemmaAnswererAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    
    # エージェントが初期化されていることを確認します
    assert agent is not None, "エージェントが初期化されていません。"

    # 初期化されたエージェントを返します
    return agent

# 観察に基づいて対話を処理する関数
def agent_fn(obs, cfg):
    # 観察が質問をするためのものである場合
    if obs.turnType == "ask":
        # 質問者エージェントを取得して観察に応じます
        response = get_agent('questioner')(obs)
    # 観察が推測をするためのものである場合
    elif obs.turnType == "guess":
        # 質問者エージェントを取得して観察に応じます
        response = get_agent('questioner')(obs)
    # 観察が回答を提供するためのものである場合
    elif obs.turnType == "answer":
        # 回答者エージェントを取得して観察に応じます
        response = get_agent('answerer')(obs)
    
    # エージェントからのレスポンスがないか、非常に短い場合
    if response is None or len(response) <= 1:
        # ポジティブな反応（「はい」）を仮定します
        return "はい"
    else:
        # エージェントから受け取ったレスポンスを返します
        return response

In [ ]:
# pigzおよびpvをインストールします（出力を抑制します）
!apt install pigz pv > /dev/null
# 圧縮されたtar.gzファイルを作成します
# submissionディレクトリ内のすべてのファイルを高圧縮で圧縮しながらtarアーカイブを作成します
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/gemma/pytorch/7b-it-quant/2

In [ ]:
# obsクラスを定義します
class obs(object):
    '''
    このクラスはゲームからの観察を表します。 
    turnTypeは['ask','answer','guess']のいずれかである必要があります。
    テストしたいキーワードを任意に選択してください。
    キーワードのカテゴリーを選択してください。
    質問は文字列のリストとして指定できます。エージェントに質問の履歴を渡す場合に使用します。
    回答も文字列のリストとして指定できます。
    responseは質問を回答者エージェントに渡すために使用します。
    '''
    def __init__(self, turnType, keyword, category, questions, answers):
        self.turnType = turnType
        self.keyword = keyword
        self.category = category
        self.questions = questions
        self.answers = answers

# エージェントに渡すobsの例。ここにあなた自身のobsを渡してください。
obs1 = obs(turnType='ask', 
          keyword='paris', 
          category='place', 
          questions=['人ですか？', '場所ですか？', '国ですか？', '都市ですか？'], 
          answers=['いいえ', 'はい', 'いいえ', 'はい'])

# エージェント関数を呼び出して質問を取得します
question = agent_fn(obs1, None)
# エージェントからのレスポンスを出力します
print(question)

# エージェントからのレスポンスが出力されるはずです。 
# これは提出時のエラーには役立ちませんが、エージェントの応答を評価するのには役立ちます。